In [3]:
import pandas as pd, numpy as np, os, math, itertools
import matplotlib.pyplot as plt
from functools import reduce
from pandarallel import pandarallel
from warnings import warn
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas() # to have df.progress_apply()
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


/home/bastiaan/.virtualenvs/phd-analysis/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [61]:
def read_simulation_samples_and_set_domains(s):
    basename = 'simulation'
    data_path = os.path.join(DATA_DIR, '{}-{}-data.csv'.format(basename, s))
    output_path = os.path.join(DATA_DIR, '{}-{}-results.csv'.format(basename, s))
    df = pd.read_csv(data_path)
    df = df.rename(columns=dict(dataset='culture'))
    df.loc[:, 'numerator'] = df.pulses.astype(int)
    # Set domains
    set_domains(df)
    return df
df = pd.read_csv('data/tutorial-data.csv')

In [60]:
TESTING_PARAMS = dict( # Format: (flags, parameters)
    viewpoint=([], dict(name='jackdaw::ioi-vp')),
    downbeat_distance=([], dict(
        outputvars=lisp_list(
            #['jackdaw::b', 'jackdaw::ioi', 'jackdaw::p', 'jackdaw::p0', 'jackdaw::period', 'jackdaw::pulses']
            ['jackdaw::ioi', 'jackdaw::p0', 'jackdaw::period', 'jackdaw::pulses']
        ))),
)
COLUMNS = dict(
    viewpoint=['ioi'],
    event_based_symbolic_temperley=['onset', 'numerator', 'denominator', 'meter'],
    downbeat_distance=['ioi', 'period', 'pulses'],
)
TRAINING_PARAMS = dict( # Format: (flags, parameters)
    viewpoint=([], dict(name='jackdaw::ioi-vp')),
    downbeat_distance=([], dict()),
    event_based_symbolic_temperley=([], dict()),
)

LATENT_VARIABLES = dict(
    downbeat_distance=['period', 'pulses', 'p0'],
    viewpoint=[],
    event_based_symbolic_temperley=['l', 'u', 't', 'bph', 'tph'])
DATA_DIR = '/home/bastiaan/data/'
DATA_ROOT = '/home/bastiaan/data/notebook/simulation_data/'
set_domains(df)

/home/bastiaan/.virtualenvs/phd-analysis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


NameError: name 'lisp_list' is not defined

In [62]:
# How many events per melody?
events = df.groupby(['dataset', 'meter', 'sequence']).event.count().reset_index()
# How many sequences per meter and dataset?
items = events.groupby(['dataset', 'meter']).sequence.count()
items.reset_index()

,dataset,meter,sequence
0,dutch,"(2, 4)",289
1,dutch,"(3, 4)",379
2,dutch,"(4, 4)",990
3,dutch,"(6, 8)",982
4,german,"(2, 4)",945
5,german,"(3, 4)",879
6,german,"(4, 4)",1323
7,german,"(6, 8)",603
8,turkish,"(10, 8)",299
9,turkish,"(4, 4)",247


In [63]:
N = items.min()
print(N)
sample = events.groupby(['dataset', 'meter']).sequence.apply(
    lambda s: s.sample(N)).reset_index()
sample.groupby(['dataset', 'meter']).sequence.count()
dataset = pd.merge(df, sample, how='inner')

232


In [54]:
# Have a look at a single rhythm
# dataset.loc[dataset.sequence == dataset.sequence.values[0], ['event', 'ioi']]
# Show score?

In [64]:
df.loc[df.event > 0]

,dataset,meter,sequence,event,ioi,usul,onset,pulses,period,denominator,fold
1,dutch,"(6, 8)",12e3edb-0,1,1,NaN,11,6,12,8,9
2,dutch,"(6, 8)",12e3edb-0,2,1,NaN,12,6,12,8,9
3,dutch,"(6, 8)",12e3edb-0,3,4,NaN,16,6,12,8,9
4,dutch,"(6, 8)",12e3edb-0,4,2,NaN,18,6,12,8,9
5,dutch,"(6, 8)",12e3edb-0,5,2,NaN,20,6,12,8,9
...,...,...,...,...,...,...,...,...,...,...,...
224275,turkish,"(4, 4)",6cb0913f2280725b-0,25,2,Sofyan,66,4,16,4,6
224276,turkish,"(4, 4)",6cb0913f2280725b-0,26,2,Sofyan,68,4,16,4,6
224277,turkish,"(4, 4)",6cb0913f2280725b-0,27,2,Sofyan,70,4,16,4,6
224278,turkish,"(4, 4)",6cb0913f2280725b-0,28,2,Sofyan,72,4,16,4,6


In [66]:
def lisp_list(l):
    return '({})'.format(' '.join(map(str, l)))

def get_meter_params(df):
    """Given a training set, obtain the associated meter parameters."""
    meter_domain = df.loc[:, ['period', 'pulses']].drop_duplicates().sort_values(
        by=['period', 'pulses']).values
    periods = [period for period, pulses in meter_domain]
    def meter_probability(m):
        period, pulses = m
        return period / sum(periods)
    meter_params = map(lambda m: lisp_list((lisp_list(m), meter_probability(m))), meter_domain)
    return lisp_list(meter_params)

In [71]:
#dataset.groupby(['dataset', 'meter']).sequence.count()
def save_dataset(df):
    dataset = df.dataset.values[0]
    df = df.loc[df.event > 0, ['sequence', 'event', 'ioi', 'period', 'pulses']]
    df.loc[:, 'event'] = df.event -1
    df = df.rename(columns={'sequence':'uid'})
    df.to_csv(f'data/{dataset}.csv', index=False)
dataset.groupby('dataset').apply(save_dataset)
dataset.groupby('dataset').apply(lambda df: '({})'.format(' '.join(map(str, sorted(df.ioi.unique())[1:])))).reset_index()
#dataset.groupby('dataset').apply(get_meter_params).values[1]

,dataset,0
0,dutch,(1 2 3 4 5 6 7 8 10 11 12 14 15 16 20 24 28 32)
1,german,(1 2 3 4 6 7 8 10 11 12 14 16 20 24)
2,turkish,(1 2 3 4 5 6 7 8 9 10 12 14 16 18 20 22 24)


In [36]:
da = pd.DataFrame([
    {'a':1, 'b': 1},
    {'a':1, 'b': 2},
    {'a':2, 'b': 1},
    {'a':2, 'b': 2},
])
da.groupby('a').apply(lambda df: print(df))

   a  b
0  1  1
1  1  2
   a  b
2  2  1
3  2  2


""
